In [1]:
from PIL import Image
from shutil import copy

import numpy as np
import os
from io import BytesIO
from keras.preprocessing.image import img_to_array

Image.MAX_IMAGE_PIXELS = None

os.chdir('..')


C:\Users\Leonardo\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
input_dir = os.path.join(os.getcwd(), 'images2')

In [3]:
os.getcwd()

'C:\\Users\\Leonardo\\forestfire'

In [4]:
def patch_maker(file, patch_size=64):
    #Function receives a file and creates the patches of 64 x 64 as a list of numpy arrays
    patches = []
    size_0 = 1920
    size_1 = 1088
    x_pos = 0
    y_pos = 0
    img = Image.open(file)
    #Applies a 8 pixels border on top to have exactly 510 patches of size 64 x 64 on the 1920 x 1088 images
    temp = Image.new(img.mode, (img.size[0], img.size[1] + 8), color=0)
    temp.paste(img, (0, 8))
    
    for i in range(int(size_0 / patch_size) * int(size_1 / patch_size)):
        if x_pos == size_0:
            x_pos = 0
            y_pos += patch_size

        patches.append(temp.crop(box=(x_pos, y_pos, x_pos + patch_size, y_pos + patch_size)).copy())

        x_pos += patch_size
            
    return patches

In [5]:
def patch_unmaker(patches, patch_size=64):
    #Function receives a list of 510 patches of size 64 x 64 and creates an 1920 x 1080 image again
    size_0 = 1920
    size_1 = 1088
    img = Image.new('RGB', (size_0, size_1), color=0)
    x_pos = 0
    y_pos = 0
    
    for i in range(len(patches)):
        if x_pos == size_0:
            x_pos = 0
            y_pos += patch_size
            img.paste(patches[i], (x_pos, y_pos))
            x_pos += patch_size
        else:
            img.paste(patches[i], (x_pos, y_pos))
            x_pos += patch_size

    
    img = img.crop(box=(0,8, size_0, size_1))
    
    return img

In [6]:
os.mkdir(os.path.join(input_dir, 'smoke'))
os.mkdir(os.path.join(input_dir, 'not_smoke'))

In [9]:
count = 0
count_smoke = 0
count_not_smoke = 0

threshold = 0.25
numberpixels = 64*64
pixels_threshold = threshold * numberpixels

for i in sorted(os.listdir(input_dir)):
    if 'mask' in str(i):
        number = i.split('.')[0].split('_')[0]
        mask_patches = patch_maker(os.path.join(input_dir,i))
        correspondent_image = "%s_original.png" %number
        image_patches = patch_maker(os.path.join(input_dir, correspondent_image))
        
        for j, k in enumerate(mask_patches):
            if (img_to_array(k).sum() / 255) > pixels_threshold:
                newfileimgname = "%s_%s.png" %(correspondent_image.split('.')[0], j)
                image_patches[j].save(os.path.join(input_dir, 'smoke', newfileimgname))
                count_smoke += 1
            else:
                newfileimgname = "%s_%s.png" %(correspondent_image.split('.')[0], j)
                image_patches[j].save(os.path.join(input_dir, 'not_smoke', newfileimgname))
                count_not_smoke += 1
                    
        if count % 500 == 0:
            print(count)
        
        count += 1

print('%s smoke 64 x 64 patches were created' %count_smoke)
print(' and %s not_smoke 64 x 64 patches were created' %count_not_smoke)

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000


In [6]:
os.mkdir(os.path.join(input_dir, 'train'))
os.mkdir(os.path.join(input_dir, 'val'))
os.mkdir(os.path.join(input_dir, 'train/smoke'))
os.mkdir(os.path.join(input_dir, 'train/not_smoke'))
os.mkdir(os.path.join(input_dir, 'val/smoke'))
os.mkdir(os.path.join(input_dir, 'val/not_smoke'))

In [7]:
def val_generator(category, percentage_images_used, train_split):
    path = os.path.join(input_dir, category)
    for i in sorted(os.listdir(path)):
        random = np.random.random()
        if random > percentage_images_used:
            pass
        else:
            random2 = np.random.random()
            if random2 > train_split:
                copy(os.path.join(path, i), (os.path.join(input_dir,'val', category)), follow_symlinks=True)
            else:
                copy(os.path.join(path, i), (os.path.join(input_dir,'train', category)), follow_symlinks=True)

In [8]:
# to replicate the work done on paper, percentage_images_used needs to be equal to 0,053
val_generator('smoke', 0.3, 0.85)

In [9]:
val_generator('not_smoke', 0.03, 0.85)